### Importing Libraries

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.layers import Input, Dense

from keras.callbacks import ModelCheckpoint, EarlyStopping

import tensorflow
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, RepeatVector, TimeDistributed
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Conv2D, BatchNormalization, Flatten, Input, Reshape, Flatten, MaxPooling2D, UpSampling2D
from tensorflow.keras.layers import BatchNormalization, Flatten
from tensorflow.keras.layers import LSTM
from tensorflow.keras import regularizers

from sklearn.metrics import classification_report, confusion_matrix
import keras
from keras import layers

from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("../input/digit-recognizer/train.csv")
df.head()

In [ ]:
df1  = pd.read_csv("../input/digit-recognizer/test.csv")
df1.head()

#### Splitting and Reshaping the dataset

In [ ]:
x_train = df.drop(['label'], axis=1).values
train_y = df['label'].values
x_test = df1.values

print("X_train shape", x_train.shape)
print("y_test shape", train_y.shape)
print("X_test shape", x_test.shape)

In [ ]:
IMG_SIZE = 32

In [ ]:
import cv2

def resize(img_array):
    tmp = np.empty((img_array.shape[0], IMG_SIZE, IMG_SIZE))

    for i in range(len(img_array)):
        img = img_array[i].reshape(28, 28).astype('uint8')
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        img = img.astype('float32')/255
        tmp[i] = img
        
    return tmp

x_train = resize(x_train)
x_test = resize(x_test)

In [ ]:
X_train_final = np.stack((x_train,)*3, axis=-1)
X_test_final = np.stack((x_test,)*3, axis=-1)
print(X_train_final.shape)
print(X_test_final.shape)

In [ ]:
y_train_final = to_categorical(train_y, num_classes=10)
print(y_train_final.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_train_final, y_train_final, test_size=0.2, random_state=2021)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
# plot first few images
import matplotlib.pyplot as plt
for i in range(9):
    plt.subplot(330 + 1 + i)
    plt.imshow(X_train[i], cmap=plt.get_cmap('gray'))
# show the figure
plt.show()

#### Code which helps to plot the Accuracy and Loss Curve

In [ ]:
def plot_history(history):
    loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' not in s]
    val_loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' in s]
    acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' not in s]
    val_acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' in s]
    
    if len(loss_list) == 0:
        print('Loss is missing in history')
        return 
    
    ## As loss always exists
    epochs = range(1,len(history.history[loss_list[0]]) + 1)
    
    ## Loss
    plt.figure(1)
    plt.figsize=(10, 10)
    for l in loss_list:
        plt.plot(epochs, history.history[l], 'b', label='Training loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    for l in val_loss_list:
        plt.plot(epochs, history.history[l], 'g', label='Validation loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    ## Accuracy
    plt.figure(2)
    plt.figsize=(10, 10)
    for l in acc_list:
        plt.plot(epochs, history.history[l], 'b', label='Training accuracy (' + str(format(history.history[l][-1],'.5f'))+')')
    for l in val_acc_list:    
        plt.plot(epochs, history.history[l], 'g', label='Validation accuracy (' + str(format(history.history[l][-1],'.5f'))+')')

    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

## Comparitive-Models

### Convolution Neural Network (CNN)

In [ ]:
model1 = Sequential()
model1.add(Conv2D(IMG_SIZE, (3, 3), activation='relu', padding='valid'))
model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(Flatten())
model1.add(Dense(512, activation='relu'))
model1.add(Dropout(0.1))
model1.add(Dense(10, activation='softmax'))
model1.compile(loss='categorical_crossentropy',
              optimizer="adam", metrics=['accuracy'])

In [ ]:
es = EarlyStopping(monitor='val_accuracy', verbose=1, patience=5)
mc = ModelCheckpoint(filepath='CNN.h5', verbose=1, monitor='val_acc')
cb = [es, mc]

In [ ]:
history1 = model1.fit(X_train, y_train, 
                    epochs=50, 
                    batch_size=128, 
                    validation_data=(X_test, y_test),
                    callbacks=cb)

In [ ]:
plot_history(history1)

In [ ]:
score1 = model1.evaluate(X_test, y_test, verbose=0)
print('Test accuracy CNN%:', (score1[1]*100))

In [ ]:
pred_1=model1.predict(X_test, batch_size=128)
y_pred_1= pred_1.argmax(axis=-1)

y_true_onehot = y_test
y_true_label = np.argmax(y_true_onehot, axis=1)
y_true = y_true_label
y_pred1 = y_pred_1
print(y_pred1[0:10])

In [ ]:
cm = confusion_matrix(y_true, y_pred1)
print(cm)

In [ ]:
plt.figure(figsize=(9,9))
plt.imshow(cm, interpolation='nearest', cmap='Pastel1')
plt.title('Confusion matrix', size = 15)
plt.colorbar()
tick_marks = np.arange(10)
plt.xticks(tick_marks, ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"], rotation=45, size = 10)
plt.yticks(tick_marks, ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"], size = 10)
plt.tight_layout()
plt.ylabel('Actual label', size = 15)
plt.xlabel('Predicted label', size = 15)
width, height = cm.shape

for x in range(width):
    for y in range(height):
        plt.annotate(str(cm[x][y]), xy=(y, x), 
                    horizontalalignment='center',
                    verticalalignment='center')
plt.savefig('CNN.png')


### Convolution Neural Network with AutoEncoder

In [ ]:
input_signal = Input(shape=(32, 32, 3))

#ENCODER
 
e = Conv2D(filters = 512, kernel_size = (3, 3), activation='relu', padding='same',  kernel_initializer='he_uniform')(input_signal)
e = MaxPooling2D(pool_size = (2, 2), padding='same')(e)
e = Dropout(0.1)(e)


e = Conv2D(filters = 256, kernel_size = (3, 3), activation='relu', padding='same',  kernel_initializer='he_uniform')(e)
e = MaxPooling2D(pool_size = (2, 2), padding='same')(e)
e = Dropout(0.1)(e)

e = Conv2D(filters = 256, kernel_size = (3, 3), activation='relu', padding='same',  kernel_initializer='he_uniform')(e)
e = MaxPooling2D(pool_size = (2, 2), padding='same')(e)
e = Dropout(0.1)(e)

##DECODER
d = Conv2D(256, (3, 3), activation='relu', padding='same',  kernel_initializer='he_uniform')(e)
d = MaxPooling2D(pool_size = (2, 2), padding='same')(d)
d = Dropout(0.1)(d)

d = Conv2D(256, (3, 3), activation='relu', padding='same',  kernel_initializer='he_uniform')(d)
d = MaxPooling2D(pool_size = (2, 2), padding='same')(d)
d = Dropout(0.1)(d)

d = Conv2D(512, (3, 3), activation='relu', padding='same',  kernel_initializer='he_uniform')(d)
d = MaxPooling2D(pool_size = (2, 2), padding='same')(d)
d = Dropout(0.1)(d)

d = Flatten()(d)
decoded = Dense(10, activation="softmax")(d)


model2 = Model(inputs=input_signal, outputs=decoded)
model2.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model2.summary()

In [ ]:
es = EarlyStopping(monitor='val_accuracy', verbose=1, patience=5)
mc = ModelCheckpoint(filepath='CNN-AutoEncoder.h5', verbose=1, monitor='val_acc')
cb = [es, mc]

In [ ]:
history2 = model2.fit(X_train, y_train, 
                    epochs=50, 
                    batch_size=128, 
                    validation_data=(X_test, y_test),
                    callbacks=cb)
scores = model2.evaluate(X_test, y_test, verbose=0)

In [ ]:
plot_history(history2)

In [ ]:
score2 = model2.evaluate(X_test, y_test, verbose=0)
print('Test accuracy CNN AutoEncoder%:', (score2[1]*100))

In [ ]:
pred_2=model2.predict(X_test, batch_size=128)
y_pred_2= pred_2.argmax(axis=-1)

y_true_onehot = y_test
y_true_label = np.argmax(y_true_onehot, axis=1)
y_true = y_true_label
y_pred2 = y_pred_2
print(y_pred2[0:10])

In [ ]:
cm = confusion_matrix(y_true, y_pred2)
print(cm)

In [ ]:
plt.figure(figsize=(9,9))
plt.imshow(cm, interpolation='nearest', cmap='Pastel2')
plt.title('Confusion matrix', size = 15)
plt.colorbar()
tick_marks = np.arange(10)
plt.xticks(tick_marks, ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"], rotation=45, size = 10)
plt.yticks(tick_marks, ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"], size = 10)
plt.tight_layout()
plt.ylabel('Actual label', size = 15)
plt.xlabel('Predicted label', size = 15)
width, height = cm.shape

for x in range(width):
    for y in range(height):
        plt.annotate(str(cm[x][y]), xy=(y, x), 
                    horizontalalignment='center',
                    verticalalignment='center')
plt.savefig('CNN-AutoEncoder.png')

### VGG-16 Transfer-Learning

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

In [ ]:
VGG = VGG16(input_shape=(32, 32, 3), include_top=False, weights='imagenet')
VGG.trainable = False

model3 = Sequential([
    VGG,
    Flatten(),
    Dense(units=512, activation="relu"),
    Dropout(0.1),
    Dense(units=256, activation="relu"),
    Dropout(0.1),
    Dense(units=10, activation="softmax")
    
])

model3.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model3.summary()

In [ ]:
es = EarlyStopping(monitor='val_accuracy', verbose=1, patience=5)
mc = ModelCheckpoint(filepath='CNN-VGG16.h5', verbose=1, monitor='val_acc')
cb = [es, mc]

In [ ]:
history3 = model3.fit(X_train, y_train, 
                    epochs=50, 
                    batch_size=128, 
                    validation_data=(X_test, y_test),
                    callbacks=cb)

In [ ]:
plot_history(history3)

In [ ]:
score3 = model3.evaluate(X_test, y_test, verbose=0)
print('Test accuracy CNN VGG-16%:', (score3[1]*100))

In [ ]:
pred_3=model3.predict(X_test, batch_size=128)
y_pred_3= pred_3.argmax(axis=-1)

y_true_onehot = y_test
y_true_label = np.argmax(y_true_onehot, axis=1)
y_true = y_true_label
y_pred3 = y_pred_3
print(y_pred3[0:10])

In [ ]:
cm = confusion_matrix(y_true, y_pred3)
print(cm)

In [ ]:
plt.figure(figsize=(9,9))
plt.imshow(cm, interpolation='nearest', cmap='Paired')
plt.title('Confusion matrix', size = 15)
plt.colorbar()
tick_marks = np.arange(10)
plt.xticks(tick_marks, ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"], rotation=45, size = 10)
plt.yticks(tick_marks, ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"], size = 10)
plt.tight_layout()
plt.ylabel('Actual label', size = 15)
plt.xlabel('Predicted label', size = 15)
width, height = cm.shape

for x in range(width):
    for y in range(height):
        plt.annotate(str(cm[x][y]), xy=(y, x), 
                    horizontalalignment='center',
                    verticalalignment='center')
plt.savefig('CNN-VGG-16.png')

### VGG-19 Transfer Learning

In [ ]:
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input

In [ ]:
VGG = VGG19(input_shape=(32, 32, 3), include_top=False, weights='imagenet')
VGG.trainable = False

model4 = Sequential([
    VGG,
    Flatten(),
    Dense(units=512, activation="relu"),
    Dropout(0.1),
    Dense(units=256, activation="relu"),
    Dropout(0.1),
    Dense(units=10, activation="softmax")
    
])

model4.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model4.summary()

In [ ]:
es = EarlyStopping(monitor='val_accuracy', verbose=1, patience=5)
mc = ModelCheckpoint(filepath='CNN-VGG19.h5', verbose=1, monitor='val_acc')
cb = [es, mc]

In [ ]:
history4 = model4.fit(X_train, y_train, 
                    epochs=50, 
                    batch_size=128, 
                    validation_data=(X_test, y_test),
                    callbacks=cb)
scores = model4.evaluate(X_test, y_test, verbose=0)
print('Test accuracy CNN-VGG19-Model%:', (scores[1]*100))

In [ ]:
plot_history(history4)

In [ ]:
score4 = model4.evaluate(X_test, y_test, verbose=0)
print('Test accuracy CNN VGG-19%:', (score4[1]*100))

In [ ]:
pred_4=model4.predict(X_test, batch_size=128)
y_pred_4= pred_4.argmax(axis=-1)

y_true_onehot = y_test
y_true_label = np.argmax(y_true_onehot, axis=1)
y_true = y_true_label
y_pred4 = y_pred_4
print(y_pred4[0:10])

In [ ]:
cm = confusion_matrix(y_true, y_pred4)
print(cm)

In [ ]:
plt.figure(figsize=(9,9))
plt.imshow(cm, interpolation='nearest', cmap='Accent')
plt.title('Confusion matrix', size = 15)
plt.colorbar()
tick_marks = np.arange(10)
plt.xticks(tick_marks, ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"], rotation=45, size = 10)
plt.yticks(tick_marks, ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"], size = 10)
plt.tight_layout()
plt.ylabel('Actual label', size = 15)
plt.xlabel('Predicted label', size = 15)
width, height = cm.shape

for x in range(width):
    for y in range(height):
        plt.annotate(str(cm[x][y]), xy=(y, x), 
                    horizontalalignment='center',
                    verticalalignment='center')
plt.savefig('CNN-VGG-19.png')

### Evaluating and SUBMISSION

As it is observed that CNN Autoencoder is giving the Highest Test Accuracy of 99.01%

We will be submtting that model

In [ ]:
from tensorflow.keras.models import load_model

model = load_model('CNN-AutoEncoder.h5')
prediction = model.predict([X_test_final])

In [ ]:
print(np.argmax(prediction[0]))

In [ ]:
plt.imshow(X_test_final[0])
plt.show

In [ ]:
label = [np.argmax(x) for x in prediction]
ImageId = [x+1 for x in range(len(prediction))]

submission = pd.DataFrame({'ImageId': ImageId,
                          'Label' : label}, index = None)
submission

In [ ]:
submission.to_csv('MySubmission.csv', index=False)